https://github.com/NationalSecurityAgency/ghidra/blob/master/Ghidra/Features/Decompiler/src/decompile/cpp/emulate.cc
https://github.com/NationalSecurityAgency/ghidra/blob/master/Ghidra/Features/Decompiler/src/decompile/cpp/opbehavior.cc#L126

In [1]:
from kdrag.all import *
import kdrag.printers.c as Cprint



x,y = smt.BitVecs("x y", 64)
so_file = Cprint.compile_c(Cprint.cstring("foo", [x,y], smt.If(smt.UGT(x + x*y + 1, x), x , y)), opts=["-O3"])
so_file


'/tmp/tmpokoye9on/temp.so'

In [2]:
!objdump -d -F /tmp/tmpd2wca2dz/temp.so


/tmp/tmpd2wca2dz/temp.so:     file format elf64-x86-64


Disassembly of section .init:

0000000000001000 <_init> (File Offset: 0x1000):
    1000:	f3 0f 1e fa          	endbr64
    1004:	48 83 ec 08          	sub    $0x8,%rsp
    1008:	48 8b 05 d1 2f 00 00 	mov    0x2fd1(%rip),%rax        # 3fe0 <__gmon_start__> (File Offset: 0x3fe0)
    100f:	48 85 c0             	test   %rax,%rax
    1012:	74 02                	je     1016 <_init+0x16> (File Offset: 0x1016)
    1014:	ff d0                	call   *%rax
    1016:	48 83 c4 08          	add    $0x8,%rsp
    101a:	c3                   	ret

Disassembly of section .plt:

0000000000001020 <.plt> (File Offset: 0x1020):
    1020:	ff 35 ca 2f 00 00    	push   0x2fca(%rip)        # 3ff0 <_GLOBAL_OFFSET_TABLE_+0x8> (File Offset: 0x3ff0)
    1026:	ff 25 cc 2f 00 00    	jmp    *0x2fcc(%rip)        # 3ff8 <_GLOBAL_OFFSET_TABLE_+0x10> (File Offset: 0x3ff8)
    102c:	0f 1f 40 00          	nopl   0x0(%rax)

Disassembly of section .plt.got:

0000000000

In [ ]:
import cle
import pypcode
class BinaryContext():
    def __init__(self, filename):
        self.pcode_cache = {}
        self.insn_cache = {}
        self.filename = filename
        self.ld = cle.loader.Loader(filename)
        self.bin_hash = None
        self.ctx = pypcode.Context("x86:LE:64:default")
    def disassemble(self, addr):
        if addr in self.insn_cache:
            return self.insn_cache[addr]
        memory = self.ld.memory.load(addr, addr, 0x128) # 128 bytes? good enough?
        for insn in self.ctx.disassemble(addr, 0).instructions:
            self.insn_cache[insn.addr] = insn
        return self.insn_cache[addr]
    def translate(self, addr, offset, max_size):
        
        
    def execute1(self, memstate, pc): ...



BinaryContext(so_file)

AttributeError: 'Loader' object has no attribute 'arch'

In [ ]:
    def execute1_ax(self, pc): # axiom schema form
        memstate = smt.Const("memstate", Memstate)
        body = self.execute1(memstate, pc)
        smt.Function("execute1_next", LoadData,  MemState, PC, PC)
        smt.Function("execute1_exec", LoadData, MemState, PC,  MemState) # Or make internalized tuple. PCMemState
        return  kd.axiom(smt.ForAll([memstate], execute1_exec(memstate, pc) == body[0])),
                kd.axiom(smt.ForAll([memstate], execute1_next(memstate, pc) == body[1]))

785 + 0 * RAX  maybe a light weight way of annotating expressions. Known fuseaways


https://github.com/angr/cle

In [3]:
import cle
ld = cle.Loader(so_file)
foo_sym = ld.find_symbol("foo")
code = ld.memory.load(foo_sym.rebased_addr, foo_sym.size)


In [23]:
hex(foo_sym.relative_addr)

'0x10f9'

In [3]:
import pypcode
ctx = pypcode.Context("x86:LE:64:default")
for ins in ctx.disassemble(code, foo_sym.rebased_addr, 0).instructions:
    print(f"{ins.addr.offset:#x}/{ins.length}: {ins.mnem} {ins.body}")
for op in ctx.translate(code, foo_sym.rebased_addr, 0).ops: #ctx.disassemble(code, foo_sym.rebased_addr, 0):
    print(pypcode.PcodePrettyPrinter.fmt_op(op))

0x4010f9/4: ENDBR64 
0x4010fd/1: PUSH RBP
0x4010fe/3: MOV RBP,RSP
0x401101/4: MOV qword ptr [RBP + -0x8],RDI
0x401105/4: MOV qword ptr [RBP + -0x10],RSI
0x401109/4: MOV RAX,qword ptr [RBP + -0x10]
0x40110d/4: ADD RAX,0x1
0x401111/5: IMUL RAX,qword ptr [RBP + -0x8]
0x401116/4: ADD RAX,0x1
0x40111a/4: CMP qword ptr [RBP + -0x8],RAX
0x40111e/2: JNC 0x401126
0x401120/4: MOV RAX,qword ptr [RBP + -0x8]
0x401124/2: JMP 0x40112a
0x401126/4: MOV RAX,qword ptr [RBP + -0x10]
0x40112a/1: POP RBP
0x40112b/1: RET 
IMARK ram[4010f9:4]
IMARK ram[4010fd:1]
unique[10c00:8] = RBP
RSP = RSP - 0x8
*[ram]RSP = unique[10c00:8]
IMARK ram[4010fe:3]
RBP = RSP
IMARK ram[401101:4]
unique[4400:8] = RBP + 0xfffffffffffffff8
unique[6800:8] = RDI
*[ram]unique[4400:8] = unique[6800:8]
IMARK ram[401105:4]
unique[4400:8] = RBP + 0xfffffffffffffff0
unique[6800:8] = RSI
*[ram]unique[4400:8] = unique[6800:8]
IMARK ram[401109:4]
unique[4400:8] = RBP + 0xfffffffffffffff0
unique[db80:8] = *[ram]unique[4400:8]
RAX = unique[db8

In [ ]:
from kdrag.contrib.pcode import unop, binop
import kdrag.theories.bitvec as bv

def executeCurrentOp(op, memstate, pc):
    opcode = op.opcode
    if op.opcode == pypcode.OpCode.IMARK:
        return memstate, fallthruOp(pc)
    elif op.opcode == pypcode.OpCode.LOAD:
        return executeLoad(op, memstate), fallthruOp(pc)
    elif op.opcode == pypcode.OpCode.STORE:
        return executeStore(op, memstate), fallthruOp(pc)
    elif op.opcode == pypcode.OpCode.BRANCH:
        return memstate, executeBranch(op, pc)
    elif op.opcode == pypcode.OpCode.CBRANCH:
        cond = executeCBranch(op, memstate)
        (addr1, pcode_pc1) = executeBranch(op, pc)
        (addr2, pcode_pc2) = fallThruOp(pc)
        return memstate, (smt.If(cond, addr1, addr2), smt.If(cond, pcode_pc1, pcode_pc2))
    elif op.opcode in unop:
        return executeUnary(op, memstate), fallthruOp(pc)
    elif op.opcode in binop:
        print(op, memstate)
        return executeBinary(op, memstate), fallthruOp(pc)
    else:
        raise NotImplementedError("Opcode not implemented: ", op.opcode)    

def executeUnary(op, memstate):
    in1 = memstate.getvalue(op.inputs[0])
    out = unop[op.opcode](in1)
    return memstate.setvalue(op.output, out)

def executeBinary(op, memstate):
    in1 = memstate.getvalue(op.inputs[0])
    in2 = memstate.getvalue(op.inputs[1])
    out = binop[op.opcode](in1, in2)
    return memstate.setvalue(op.output, out)

def executeLoad(op, memstate):
    off = memstate.getvalue(op.inputs[1]) # offset to load from
    spc = memstate.getvalue(op.inputs[0]) # memory space
    return memstate.setvalue(op.output, memstate.getvalue_ram(off))

def executeStore(op, memstate):
    val = memstate.getvalue(op.inputs[2]) # value being stored
    off = memstate.getvalue(op.inputs[1]) # offset to store at
    spc = memstate.getvalue(op.inputs[0]) # memory space
    print(spc)
    memstate.setvalue_ram(off, val)

def fallthruOp(pc):
    addr, pcode_pc = pc
    pcode_pc += 1
    if pcode_pc >= len(pcode_cache[addr][1]):
        addr += pcode_cache[addr][0].length
        return (addr, 0)
    else:
        return (addr, pcode_pc)

def executeBranch(op, pc):
    addr, pcode_pc = pc
    destaddr = op.inputs[0]
    if destaddr.space.name == "constant":
        pcode_pc += destaddr.offset
        if pcode_pc == len(pcode_cache[addr][1]):
            fallthruOp((addr, pcode_pc))
        elif pcode_pc < 0 or pcode_pc >= len(pcode_cache[addr][1]):
            raise ValueError(f"Lowlevel Error. Bad intra-instruction branch: {pcode_pc}")
        else:
            return (addr, pcode_pc)
    elif destaddr.space.name == "ram":
        return (destaddr.offset, 0)
    else:
        raise ValueError(f"Unknown branch target: {destaddr.space.name}")

def executeCbranch(op, memstate):
    cond = memstate.getvalue(op.inputs[1])
    return cond != 0

MemSort = smt.ArraySort(smt.BitVecSort(64), smt.BitVecSort(8))
MemStateSort = kd.Struct("MemState", ("ram" , MemSort), ("register", MemSort), ("unique", MemSort))

from dataclasses import dataclass
@dataclass
class MemState():
    mem : smt.DatatypeRef
    def __init__(self, mem):
        self.mem = mem
    def getvalue(self, vnode):
        if vnode.space.name == "const":
            return vnode.offset
        else:
            mem = getattr(self.mem, vnode.space.name)
            if mem is None:
                raise ValueError(f"Unknown memory space: {vnode.space.name}")
            return bv.SelectConcat(mem, vnode.offset, vnode.size)
    def setvalue(self, vnode, value):
        assert vnode.space.name != "const"
        return MemState(self.mem._replace(**{vnode.space.name: bv.StoreConcat(getattr(self.mem, vnode.space.name), addr, value)}))
    def setvalue_ram(self, offset, value):
        return MemState(self.mem._replace(ram = bv.StoreConcat(self.mem.ram, offset, value)))



In [22]:

pcode_cache = {}
def execute1(code, rebased_addr, offset):
    insn = ctx.disassemble(code, rebased_addr, offset, max_instructions=1).instructions[0]
    print(f"{insn.addr.offset:#x}/{insn.length}: {insn.mnem} {insn.body}")
    ops = ctx.translate(code, rebased_addr, offset, max_bytes=insn.length).ops
    pcode_cache[rebased_addr] = insn, ops
    pc = (rebased_addr, 0)
    memstate = MemState(smt.Const("mem", MemStateSort))
    addr = rebased_addr
    for op in ops:
        print(pypcode.PcodePrettyPrinter.fmt_op(op))
        if op.opcode == pypcode.OpCode.IMARK:
            addr = op.inputs[0].offset
            pcode_pc = 0
        memstate, pc = executeCurrentOp(op, memstate, pc)
        print(memstate, pc)
    return memstate, pc

execute1(code, foo_sym.rebased_addr, 4)

0x4010f9/1: PUSH RBP
IMARK ram[4010f9:1]
MemState(mem=mem) (4198649, 1)
unique[10c00:8] = RBP
MemState(mem=MemState(ram(mem),
         register(mem),
         Store(Store(Store(Store(Store(Store(Store(Store(unique(mem),
                                        4198649,
                                        Extract(7,
                                        0,
                                        Concat(Concat(Concat(Concat(Concat(Concat(Concat(register(mem)[47],
                                        register(mem)[46]),
                                        register(mem)[45]),
                                        register(mem)[44]),
                                        register(mem)[43]),
                                        register(mem)[42]),
                                        register(mem)[41]),
                                        register(mem)[40]))),
                                        4198650,
                                        Extract(15,
      

(None, (4198650, 0))

NameError: name 'cle' is not defined

In [ ]:
    def sym_execute(self, memstate, addr, trace=None, max_insns=1):
        pc = (addr, 0)
        todo = [(memstate, pc, max_insns)]
        while True:
            memstate, pc, max_insns = todo.pop()
            memstate, pc1 = self.execute1(memstate, addr)
            addr1, pcode_pc1 = pc1
            for models in smt.solve(addr1, pcodepc1):
                # if isnstance(addr, smt.ExprRef) or isinstance(pcode_pc1, smt.ExprRef): models = [(addr, addr1)
                
            #if not smt.is_value(addr1) or not smt.is_value(pcode_pc1): # maybe faster optimization.
            #   # branch 

                if pc1[0] != pc[0]:
                    max_insns -= 1
                if max_insns > 0:
                    todo.append((memstate, pc1, max_insns))

You know, readability wise, we'd be better off using an array(64,64)

concat(  ,  , select(select(addr >> 2)))

Cleanness will depend on whether common case is word or half word vs byte manipulations.
Byte miauplations oinly pay 3 junk, but word manipulations pay 8 junk.
Byte manipulatin doesn't mangle the constants




If we don't get a conrete address, use finite model expansion to figure out all possible addresses


In [ ]:

addr = foo_sym.rebased_addr  
pcode_pc = 0  
memstate = MemState(smt.Const("mem", MemStateSort))


"""
for op in ctx.translate(code, foo_sym.rebased_addr, max_intructions=1).ops:
    pcode_pc += 1
    if op.opcode == pypcode.OpCode.IMARK:
        addr = op.inputs[0].offset
        pcode_pc = 0
    print(pypcode.PcodePrettyPrinter.fmt_op(op))
    executeCurrentOp(op, memstate, (addr, pcode_pc))
"""



In [46]:
import functools
import pypcode
unop = {
    OpCode.COPY : lambda x: x,
    OpCode.INT_2COMP : operator.neg,
    OpCode.INT_NEGATE : operator.invert,
    OpCode.BOOL_NEGATE : lambda x: smt.If(x == TRUE, FALSE, TRUE),
}

binop = {
    OpCode.INT_ADD: operator.add,
    OpCode.INT_SUB: operator.sub,


    OpCode.INT_XOR: operator.xor,
    OpCode.INT_AND: operator.and_,
    OpCode.INT_OR: operator.or_,
    OpCode.INT_LEFT: operator.lshift,
    OpCode.INT_RIGHT: smt.LShR,
    OpCode.INT_SRIGHT: operator.rshift,
    OpCode.INT_MULT: operator.mul,
    OpCode.INT_DIV: smt.UDiv,
    OpCode.INT_REM: smt.URem,
    OpCode.INT_SDIV: operator.truediv,
    OpCode.INT_SREM: smt.SRem,

    OpCode.BOOL_XOR: lambda x,y: smt.If(x == y, FALSE, TRUE),
    OpCode.BOOL_AND: operator.and_,
    OpCode.BOOL_OR: operator.or_,
}

comp = {
    OpCode.INT_EQUAL: operator.eq,
    OpCode.INT_NOTEQUAL: operator.ne,
    OpCode.INT_LESS: smt.ULT,
    OpCode.INT_SLESS: operator.lt,
    OpCode.INT_LESSEQUAL: smt.ULE,
    OpCode.INT_SLESSEQUAL: operator.le,
    OpCode.INT_CARRY: None, # TODO
    OpCode.INT_SCARRY: None, # TODO
    OpCode.INT_SBORROW: None, # TODO
}


@functools.cache
def REG(name, offset):
    return kd.define(name, [], smt.BitVecVal(offset, 64))

def z3_of_varnode(vnode, state):
    print(vnode.space.name)
    if vnode.space.name == "const":
        return vnode.offset
    elif vnode.space.name == "register":
        return bv.SelectConcat(getattr(state, vnode.space.name), REG(vnode.getRegisterName(), vnode.offset), vnode.size)
    else:
        #print(vnode.space.name)
        return bv.SelectConcat(getattr(state, vnode.space.name), vnode.offset, vnode.size)

def store(state, vnode, value):
    if vnode.space.name == "register":
        return state._replace(register=smt.Store(state.register, REG(vnode.getRegisterName(), vnode.offset), value))
    elif vnode.space.name == "ram":
        return state._replace(ram=smt.Store(state.ram, vnode.offset, value))
    elif vnode.space.name == "unique":
        return state._replace(ram=smt.Store(state.unique, vnode.offset, value))
    else:
        raise NotImplementedError("space not implemented", vnode.space.name)

def z3_of_op(op, state):
    opcode = op.opcode
    if opcode == OpCode.IMARK:
        return state
    elif opcode in binop:
        #assert isinstance(opcode, pypcode.OpFormatBinary)
        src1 = z3_of_varnode(op.inputs[0], state)
        src2 = z3_of_varnode(op.inputs[1], state)
        return store(state, op.output, binop[op.opcode](src1, src2))
    elif opcode in unop:
        src = z3_of_varnode(op.inputs[0], state)
        return store(state, op.output, unop[op.opcode](src))
    elif opcode in comp:
        src1 = z3_of_varnode(op.inputs[0], state)
        src2 = z3_of_varnode(op.inputs[1], state)
        return store(state, op.output, smt.If(comp[op.opcode](src1,src2), TRUE, FALSE))
    else:
        raise NotImplementedError("opcode not implemented", op.opcode)


In [47]:
tx = ctx.translate(b"\xb8\x05\x00\x00\x00\xbb\x0a\x00\x00\x00\x8d\x0c\x18\x0f\xaf\xd8"
)#b"\xb8\x05\x00\x00\x00")
state = smt.Const("init", State)
for op in tx.ops:
    print(PcodePrettyPrinter.fmt_op(op))
    state = z3_of_op(op, state)
print(state)

IMARK ram[0:5]
RAX = 0x5
const
IMARK ram[5:5]
RBX = 0xa
const
IMARK ram[a:3]
unique[4600:8] = RBX * 0x1
register
const


Z3Exception: b'domain sort (_ BitVec 64) and parameter sort (_ BitVec 8) do not match'

In [ ]:
PC = smt.BitVec("PC", 64)
state = smt.Const("state", State)
undef = smt.Function("undef", smt.BitVecSort(64), State)
fetch = smt.Function("fetch", smt.BitVecSort(64), State, State)
fetch = kd.define("fetch", [PC, op_num, state],
    kd.cond(
(PC == 0,  insn_0(state))
        default=undef(PC)
    )              
    )




symbolic execution. metalayer PC opnum. State is symbolic.
deep embed pcode instructions. deep embed interpreter.
Specialize to particular program.
I can also specialize the fields. to flip them up to the metalayer.
I don't have to reveal all at once.

There could be an axiom schema for the program.


discovery of control flow is part of proof burden


lazy every possible address, kind of like datalog disassembly


is it ok that we can have non terminating loops?

Well. actually both of these are in just pure "data" forms. The fixed point hasn't been asserted.


In [ ]:
executeCurrentOp = kd.define("executeCurrentOp", [PC, opnum, state], state)
def executecurrentOp(PC, opnum, op):
    insn = pypcode.tx(addr)
    insn.ops[opnum]

@functools.cache
def exec_ax(addr, opnum):
    op = tx.translate(addr,opnum)
    kd.axiom(smt.ForAll([state], exec(PC, opnum, state) == z3_of_op(op, state)))
    if is_fallthrough(op):
        if opnum+1 < len(tx.translate(addr)):
            kd.axiom(smt.ForAll([state], next(PC, opnum, state) == (addr, opnum+1)))
        else:
            kd.axiom(smt.ForAll([state], next(PC, opnum, state) == (addr+len(tx.translate(addr)), 0)))
    else:
        if op.opcde == OpCode.BRANCH:
            kd.axiom(smt.ForAll([state], next(PC, opnum, state) == (op.inputs[0].offset, 0)))


interp(PC, opnum, state, breakpoints) == 
    smt.If(PC == breakpoint, state, 
           interp(next(PC,opnum, state),exec(PC, opnum, state)))


State = kd.Struct(("addr", ), 


In [32]:
    """
    elif opcode == OpCode.STORE:
        raise NotImplementedError("opcode not implemented", op.opcode)
    elif opcode == OpCode.LOAD:
        addr = z3_of_varnode(op.inputs[1], state)
        return bv.SelectConcat(state.ram, addr, op.output.size)
    elif opcode == OpCode.BRANCH:
        if op.inputs[0].space.name == "ram":
            return smt.Store(state.register, pc, op.inputs[0].offset)
        else:
            raise NotImplementedError("unsupported pcode realtive", op)
    elif opcode == OpCode.CBRANCH:
        if op.inputs[0].space.name == "ram":
            cond = z3_of_varnode(op.inputs[1], state)
            return smt.If(cond, store(state, pc, op.inputs[0].offset), state)
    """


#def insn

#def trans_rel(pc : pypcode.VarNode, ops):



'\nelif opcode == OpCode.STORE:\n    raise NotImplementedError("opcode not implemented", op.opcode)\nelif opcode == OpCode.LOAD:\n    addr = z3_of_varnode(op.inputs[1], state)\n    return bv.SelectConcat(state.ram, addr, op.output.size)\nelif opcode == OpCode.BRANCH:\n    if op.inputs[0].space.name == "ram":\n        return smt.Store(state.register, pc, op.inputs[0].offset)\n    else:\n        raise NotImplementedError("unsupported pcode realtive", op)\nelif opcode == OpCode.CBRANCH:\n    if op.inputs[0].space.name == "ram":\n        cond = z3_of_varnode(op.inputs[1], state)\n        return smt.If(cond, store(state, pc, op.inputs[0].offset), state)\n'

In [19]:
from pypcode import Context, PcodePrettyPrinter, PcodeOp, OpCode, Varnode
import z3
from kdrag.all import *
import kdrag.theories.bitvec as bv
import operator
#State = smt.DeclareSort("State")

def z3_of_varnode(vnode, state):
    if vnode.space.name == "const":
        return vnode.offset
    else:
        #print(vnode.space.name)
        return bv.SelectConcat(getattr(state, vnode.space.name), vnode.offset, vnode.size)

BV8 = bv.BitVecSort(8)
BV64 = bv.BitVecSort(64)

TRUE = smt.BitVecVal(1, 8)
FALSE = smt.BitVecVal(0, 8)

def BoolToBV8(x):
    return smt.If(x, TRUE, FALSE)



def z3_of_op(op, state):
    inputs = [z3_of_varnode(v, state) for v in op.inputs]

    match op.opcode:
        case OpCode.COPY:
            src = z3_of_varnode(op.inputs[0], state)
            return src
        case OpCode.INT_ADD:
            src1 = z3_of_varnode(op.inputs[0], state)
            src2 = z3_of_varnode(op.inputs[1], state)
            return src1 + src2
        case OpCode.INT_XOR:
            src1 = z3_of_varnode(op.inputs[0], state)
            src2 = z3_of_varnode(op.inputs[1], state)
            return src1 ^ src2
        case OpCode.INT_SLESS:
            src1 = z3_of_varnode(op.inputs[0], state)
            src2 = z3_of_varnode(op.inputs[1], state)
            return BoolToBV8(src1 < src2)
        case OpCode.INT_EQUAL:
            src1 = z3_of_varnode(op.inputs[0], state)
            src2 = z3_of_varnode(op.inputs[1], state)
            return BoolToBV8(src1 == src2)
        case OpCode.INT_AND:
            src1 = z3_of_varnode(op.inputs[0], state)
            src2 = z3_of_varnode(op.inputs[1], state)
            return src1 & src2
        case OpCode.POPCOUNT:
            src = z3_of_varnode(op.inputs[0], state)
            popcount = smt.Function("popcount", BV64, BV8)
            return popcount(src)
        case OpCode.LOAD:
            addr = z3_of_varnode(op.inputs[1], state)
            return bv.SelectConcat(state.ram, addr, op.output.size) # todo: where does that length come from? output node?
        case OpCode.IMARK:
            return None
        case OpCode.RETURN:
            return None
        case _:
            raise NotImplementedError("opcode not implemented", op.opcode)




State = kd.Struct("State", ("ram", smt.ArraySort(BV64, BV8)), 
                          ("register", smt.ArraySort(BV64, BV8)),
                          ("unique", smt.ArraySort(BV64, BV8)))


#smt.ArraySort(smt.IntSort(), smt.ArraySort(smt.BitVecSort(64), smt.BitVecSort(8)))#smt.DeclareSort("State")
state = smt.Const("init_state", State)
ctx = Context("x86:LE:64:default")
tx = ctx.translate(b"\x48\x35\x78\x56\x34\x12\xc3")
for op in tx.ops:
    print(PcodePrettyPrinter.fmt_op(op))
    print(z3_of_op(op, state))

IMARK ram[0:6]
None
CF = 0x0
0
OF = 0x0
0
RAX = RAX ^ 0x12345678
Concat(Concat(Concat(Concat(Concat(Concat(Concat(register(init_state)[7],
                                        register(init_state)[6]),
                                        register(init_state)[5]),
                                   register(init_state)[4]),
                            register(init_state)[3]),
                     register(init_state)[2]),
              register(init_state)[1]),
       register(init_state)[0]) ^
305419896
SF = RAX s< 0x0
If(Concat(Concat(Concat(Concat(Concat(Concat(Concat(register(init_state)[7],
                                        register(init_state)[6]),
                                        register(init_state)[5]),
                                      register(init_state)[4]),
                               register(init_state)[3]),
                        register(init_state)[2]),
                 register(init_state)[1]),
          register(init_state)[0]) <
   0,
 

In [10]:
%%file /tmp/foo.c

int foo(int x) {
    //assert(x == 0x12345678);
    {
        int b = 4;
    }
    {
        int b = 64;
    }
    int y = x + 42;
    //int y = 14;
    for(int i = 0; i < 10; i++) {
        y = y * 2;
    }
    //y = y * 15;
    int z = y * 3;
    return z;
}

Overwriting /tmp/foo.c


(get-value (|foo::1::y!0@1#1|))
(get-value (|foo::1::y!0@1#2|))
(get-value (|foo::1::y!0@1#3|))
(get-value (|foo::1::y!0@1#4|))
(get-value (|foo::1::y!0@1#5|))
(get-value (|foo::1::y!0@1#6|))
(get-value (|foo::1::y!0@1#7|))
(get-value (|foo::1::y!0@1#8|))
(get-value (|foo::1::y!0@1#9|))
(get-value (|foo::1::z!0@1#1|))
(get-value (|foo::x!0@1#1|))
(get-value (|nondet_symex::nondet0|))
(get-value (|symex::args::0|))

This is some interpretable stuff in the variable names. But it isn't super straightforward.
I can't dig out the return value. COuld do an external assert maybe

# capstone


captsone interpreter
https://www.capstone-engine.org/

In [2]:
%%file /tmp/test.c

int myadd(int a, int b) {
    return a + b;
}


Writing /tmp/test.c


In [3]:
! gcc -c -o /tmp/test.so /tmp/test.c

In [4]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"

md = Cs(CS_ARCH_X86, CS_MODE_64)
for i in md.disasm(CODE, 0x1000):
    print("0x%x:\t%s\t%s" %(i.address, i.mnemonic, i.op_str))


0x1000:	push	rbp
0x1001:	mov	rax, qword ptr [rip + 0x13b8]


In [ ]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"
from collections import defaultdict
class State():
    mem = defaultdict(None)
    reg = defaultdict(None)

def interp(op, state):
    
    match op.mnemonic:
        case "add":
            state.reg[op.operands[0].reg] += state[op.operands[1].reg]
        case "sub":
            state[op.operands[0].reg] -= state[op.operands[1].reg]
        case "mov":
            if op.operands[0].type == X86_OP_IMM:
                state[op.operands[0].reg] = op.operands[1].imm
            elif op.operands[0].type == X86_OP_REG:
                op.operands[0].reg = op.operands[1].reg
            elif op.operands[0].type == X86_OP_MEM:
                state[op.operands[0].reg] = mem[state[op.operands[1].reg + op.operands[1].disp]
    return state
state = defaultdict(None)
for i in md.disasm(CODE, 0x1000):
    print(dir(i))
    interp(i, state)


['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs', '_raw', 'address', 'bytes', 'errno', 'group', 'group_name', 'groups', 'id', 'insn_name', 'mnemonic', 'op_count', 'op_find', 'op_str', 'reg_name', 'reg_read', 'reg_write', 'regs_access', 'regs_read', 'regs_write', 'size']
['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs',

CsError: Details are unavailable (CS_ERR_DETAIL)

Ok so it gets hairier and harrier

Getting the smeanuitcs



# Pcode

In [ ]:
import pypcode
class State():
    space = defaultdict(defaultdict)

def interp(insn, state):
    match insn.op:
        case pypcode.PcodeOp.INT_ADD:
            


In [1]:
%%file /tmp/foo.c
int foo(int x, int y){
    return x > y ? x : y;
}

Writing /tmp/foo.c


In [2]:
%%bash
gcc /tmp/foo.c -c -o /tmp/foo.o
objdump -d -F /tmp/foo.o


/tmp/foo.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <foo> (File Offset: 0x40):
   0:	f3 0f 1e fa          	endbr64 
   4:	55                   	push   %rbp
   5:	48 89 e5             	mov    %rsp,%rbp
   8:	89 7d fc             	mov    %edi,-0x4(%rbp)
   b:	89 75 f8             	mov    %esi,-0x8(%rbp)
   e:	8b 55 f8             	mov    -0x8(%rbp),%edx
  11:	8b 45 fc             	mov    -0x4(%rbp),%eax
  14:	39 c2                	cmp    %eax,%edx
  16:	0f 4d c2             	cmovge %edx,%eax
  19:	5d                   	pop    %rbp
  1a:	c3                   	ret    


In [3]:
import pypcode
# https://github.com/philzook58/pcode2c/blob/main/pcode2c/printer.py
from pypcode import Context, PcodePrettyPrinter
ctx = Context("x86:LE:64:default")
file_offset = 0x40
filename = "/tmp/foo.o"
size = 0x1a
start_address = 0
with open(filename, "rb") as file:
    file.seek(file_offset)
    code = file.read(size)
res = ctx.translate(code, base_address=start_address)
for op in res.ops:
    print(PcodePrettyPrinter.fmt_op(op))

IMARK ram[0:4]
IMARK ram[4:1]
unique[10c00:8] = RBP
RSP = RSP - 0x8
*[ram]RSP = unique[10c00:8]
IMARK ram[5:3]
RBP = RSP
IMARK ram[8:3]
unique[4400:8] = RBP + 0xfffffffffffffffc
unique[6780:4] = EDI
*[ram]unique[4400:8] = unique[6780:4]
IMARK ram[b:3]
unique[4400:8] = RBP + 0xfffffffffffffff8
unique[6780:4] = ESI
*[ram]unique[4400:8] = unique[6780:4]
IMARK ram[e:3]
unique[4400:8] = RBP + 0xfffffffffffffff8
unique[da80:4] = *[ram]unique[4400:8]
EDX = unique[da80:4]
RDX = zext(EDX)
IMARK ram[11:3]
unique[4400:8] = RBP + 0xfffffffffffffffc
unique[da80:4] = *[ram]unique[4400:8]
EAX = unique[da80:4]
RAX = zext(EAX)
IMARK ram[14:2]
unique[27c00:4] = EDX
CF = unique[27c00:4] < EAX
OF = sborrow(unique[27c00:4], EAX)
unique[27d00:4] = unique[27c00:4] - EAX
SF = unique[27d00:4] s< 0x0
ZF = unique[27d00:4] == 0x0
unique[15080:4] = unique[27d00:4] & 0xff
unique[15100:1] = popcount(unique[15080:4])
unique[15180:1] = unique[15100:1] & 0x1
PF = unique[15180:1] == 0x0
IMARK ram[16:3]
unique[eb00:1] = 

In [73]:
BV8  = BitVecSort(8)
BV16 = BitVecSort(16)
BV32 = BitVecSort(32)
BV64 = BitVecSort(64)

Select16 = Function("Select16", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(16))
Select32 = Function("Select32", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(32))
Select64 = Function("Select64", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(64))
def MSelect(a, i, n):
    if n == 1:
        return a[i]
    if n == 2:
        return Select16(a, i)
    if n == 4:
        return Select32(a, i)
    if n == 8:
        return Select64(a, i)
    else:
        assert False, "n must be 1, 2, 4 or 8"
Store16 = Function("Store16", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(16), ArraySort(BitVecSort(64), BitVecSort(8)))
Store32 = Function("Store32", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(32), ArraySort(BitVecSort(64), BitVecSort(8)))
Store64 = Function("Store64", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(64), ArraySort(BitVecSort(64), BitVecSort(8)))

def MStore(a,i,v):
    size = v.size()
    if size == 1:
        return Store(a, i, v)
    if size == 2:
        return Store16(a, i, v)
    if size == 4:
        return Store32(a, i, v)
    if size == 8:
        return Store64(a, i, v)
    else:
        assert False, "size must be 1, 2, 4 or 8"

In [14]:
state[8,2]

Select(state, 8, 2)

In [7]:
from pypcode import OpCode
from z3 import *
Space = StringSort()
Varnode = Datatype('Varnode')
Varnode.declare('v', ('space', Space), ('offset', BitVecSort(64)))
Varnode = Varnode.create()

State = ArraySort(Varnode, BitVecSort(8))
#Store("constant", Lambda([x], x))
x = BitVec("x", 64)
State0 = {"ram" : None, "unique" : None, "const" : Lambda([x], x)}

def varnode(vnode, ):
    if vnode == None:
        return None
    elif vnode.space.name == "const":
        return BitVecVal(vnode.offset, vnode.size * 8)
    return BitVecVal(vnode.offset, vnode.size * 8)

def step(op, state1, state2):
    output = varnode(op.output)
    inputs = [varnode(i) for i in op.inputs]
    match op.opcode:
        case OpCode.IMARK:
            return f"IMARK "
        case OpCode.COPY:
            return MStore(state1, output, state1[inputs[0]])
        case OpCode.INT_SUB:
            return MStore(state1, output, state1[inputs[0]] - state1[inputs[1]])
        case _:
            print("unrecognized opcode", op.opcode)
            assert False
state1, state2 = Consts("state1 state2", State)
for op in res.ops:
    print(step(op, state1, state2))

IMARK 
IMARK 


Z3Exception: Sort mismatch

https://spinsel.dev/assets/2020-06-17-ghidra-brainfuck-processor-1/ghidra_docs/language_spec/html/pcoderef.html

In [77]:
from z3 import *
from collections import namedtuple
def MStore(a,i,v):
    size = v.size()
    assert size in [8,16,32,64]
    select = Function(f"select{size}", MemSort, BitVecSort(64), BitVecSort(size), MemSort)
    return select(a,i,v)
def MSelect(a, i, n):
    assert size in [8,16,32,64]
    select = Function(f"select{n}", MemSort, BitVecSort(64), BitVecSort(n))
    return select(a,i)

def varnode(vnode, state):
    if vnode == None:
        return None
    if vnode.space.name == "const":
        print(vnode.size)
        return BitVecVal(vnode.offset, vnode.size * 8)
    v = BitVecVal(vnode.offset, 64)
    if vnode.space.name == "ram":
        return MSelect(state.ram, v, vnode.size*8)
    elif vnode.space.name == "unique":
        return MSelect(state.unique, v, vnode.size*8)
    elif vnode.space.name == "register":
        return MSelect(state.register,vnode.offset, vnode.size*8)
    else:
        raise ValueError("unrecognized space", vnode.space.name)
def step(op, state1):
    control = None
    output = None
    if op.opcode == OpCode.IMARK:
        return f"IMARK"
    elif op.opcode == OpCode.BRANCH:
        control = True
        return "branch"
    elif op.opcode == OpCode.CBRANCH:
        control = inputs[0] != 0
        return "cbranch"
    elif op.opcode == OpCode.CALL:
        return "call"
    elif op.opcode == OpCode.CALLIND:
        return "callind"
    elif op.opcode == OpCode.RETURN:
        return "return"
    
    inputs = [varnode(i,state) for i in op.inputs]
    if op.opcode == OpCode.STORE:
        #assert op.inputs[0].space.name == "ram"
        return state1._replace(ram = MStore(state1.ram, inputs[1], inputs[2]))
    elif op.opcode == OpCode.LOAD:
        #assert op.inputs[0].space.name == "ram"
        return MSelect(state.ram, inputs[1], 8)
    else:
        output = arith_op(op, inputs)
    
    

def arith_op(op, inputs):
    match op.opcode:
        case OpCode.COPY:
            return inputs[0]
        case OpCode.INT_EQUAL:
            return If(inputs[0] == inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_NOTEQUAL:
            return If(inputs[0] != inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_LESS:
            return If(ULT(inputs[0], inputs[1]), BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_SLESS:
            return If(inputs[0] < inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_LESSEQUAL:
            return If(ULE(inputs[0], inputs[1]), BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_SLESSEQUAL:
            return If(inputs[0] <= inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_ZEXT: 
            return ZeroExt((op.output.size - op.inputs[0].size) * 8, inputs[0])
        case OpCode.INT_SEXT:
            return SignExt((op.output.size - op.inputs[0].size) * 8, inputs[0])
        case OpCode.INT_ADD:
            return inputs[0] + inputs[1]
        case OpCode.INT_SUB:
            return inputs[0] - inputs[1]
        case OpCode.INT_CARRY:
            assert If(BVAddNoOverflow(inputs[0], inputs[1], False), BitVecVal(0, 8), BitVecVal(1, 8))
        case OpCode.INT_SCARRY:
            assert If(BVAddNoOverflow(inputs[0], inputs[1], True), BitVecVal(0, 8), BitVecVal(1, 8))
        case OpCode.INT_SBORROW:
            assert False, "unimplemented"
        case OpCode.INT_2COMP:
            return -inputs[0]
        case OpCode.INT_NEGATE:
            return ~inputs[0]
        case OpCode.INT_XOR:
            return inputs[0] ^ inputs[1]
        case OpCode.INT_AND:
            return inputs[0] & inputs[1]
        case OpCode.INT_OR:
            return inputs[0] | inputs[1]
        case OpCode.INT_LEFT:
            return inputs[0] << inputs[1]
        case OpCode.INT_RIGHT:
            return  LShR(inputs[0], inputs[1])
        case OpCode.INT_SRIGHT:
            return inputs[0] >> inputs[1]
        case OpCode.INT_MULT:
            return inputs[0] * inputs[1]
        case OpCode.INT_DIV:
            return UDiv(inputs[0], inputs[1])
        case OpCode.INT_REM:
            return URem(inputs[0], inputs[1])
        case OpCode.INT_SDIV:
            return inputs[0] / inputs[1]
        case OpCode.INT_SREM:
            return inputs[0] % inputs[1]
        case OpCode.BOOL_NEGATE:
            return If(inputs[0] == 0, BitVecVal(1, 8), BitVecVal(0, 8))       
        case OpCode.BOOL_XOR:
            return If(inputs[0] != inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.BOOL_AND:
            return If(inputs[0] & inputs[1] == 1, BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.POPCOUNT:
            return BitVecVal(0, output.size*8)
            for i in range(inputs[0].size()):
                output += (inputs[0] >> i) & 1
        case _:
            raise ValueError("unrecognized opcode", op.opcode)
    return output
State = namedtuple("state", "ram unique register")
MemSort = ArraySort(BitVecSort(64), BitVecSort(8))
state = State(Const("ram", MemSort), Const("unique", MemSort), Const("register", MemSort))
for op in res.ops:
    print(PcodePrettyPrinter.fmt_op(op))
    print(step(op, state))

IMARK ram[0:4]
IMARK
IMARK ram[4:1]
IMARK
unique[10c00:8] = RBP


AssertionError: 

In [36]:
def varnode(vnode):
    if vnode == None:
        return None
    v = BitVecVal(vnode.offset, vnode.size * 8)
    if vnode.space.name == "const":
        return v
    elif vnode.space.name == "ram":
        return BitVec("ram", vnode.size*8)
    elif vnode.space.name == "unique":
        return BitVec("unique", vnode.size*8)
    elif vnode.space.name == "register":
        return BitVec(f"register_{vnode.offset}", vnode.size*8)
    else:
        raise ValueError("unrecognized space", vnode.space.name)
def step(op):
    inputs = [varnode(i) for i in op.inputs]
    match op.opcode:
        case OpCode.IMARK:
            return f"IMARK "
        case OpCode.COPY:
            return inputs[0]
        case OpCode.INT_ADD:
            return inputs[0] + inputs[1]
        case OpCode.INT_SUB:
            return inputs[0] - inputs[1]
        case OpCode.STORE:
            pass
        case _:
            print("unrecognized opcode", op.opcode)
            assert False
for op in res.ops:
    print(step(op))

    print(PcodePrettyPrinter.fmt_op(op))

IMARK 
IMARK ram[0:4]
IMARK 
IMARK ram[4:1]
register_40
unique[10c00:8] = RBP
register_32 - 8
RSP = RSP - 0x8
None
*[ram]RSP = unique[10c00:8]
IMARK 
IMARK ram[5:3]
register_32
RBP = RSP
IMARK 
IMARK ram[8:3]
register_40 + 18446744073709551612
unique[4400:8] = RBP + 0xfffffffffffffffc
register_56
unique[6780:4] = EDI
None
*[ram]unique[4400:8] = unique[6780:4]
IMARK 
IMARK ram[b:3]
register_40 + 18446744073709551608
unique[4400:8] = RBP + 0xfffffffffffffff8
register_48
unique[6780:4] = ESI
None
*[ram]unique[4400:8] = unique[6780:4]
IMARK 
IMARK ram[e:3]
register_40 + 18446744073709551608
unique[4400:8] = RBP + 0xfffffffffffffff8
unrecognized opcode OpCode.LOAD


AssertionError: 

In [80]:
from pwn import asm, context
from pypcode import Context, PcodePrettyPrinter

# Set the pwntools context to RISC-V 32-bit
context.arch = 'riscv32'
context.endian = 'little'

# Define a list of simple RISC-V instructions
instructions = [
    "li t0, 42",        # Load immediate
    "add t1, t0, t0",   # Add t0 + t0 -> t1
    "sub t2, t1, t0",   # Subtract t1 - t0 -> t2
    "lw t3, 0(t0)",     # Load word from memory
    "sw t3, 4(t0)"      # Store word to memory
]
# Assemble the instructions into bytes
assembled_code = b"".join(asm(inst) for inst in instructions)

[!] Could not find system include headers for riscv32-linux
